In [83]:
#Here we are going to import all the libraries
from typing import TypedDict , List, Optional
from langchain_core.messages import BaseMessage
from langgraph.graph import StateGraph, START, END
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
import os
from google.colab import userdata
from sentence_transformers import SentenceTransformer
import numpy as np
import json

In [82]:
#This is model API key
os.environ["HUGGINGFACEHUB_API_TOKEN"] = userdata.get("HUGGINGFACEHUB_API_TOKEN")

In [81]:
#The model we are going to use
llm = HuggingFaceEndpoint(
    repo_id="Qwen/Qwen2.5-7B-Instruct",
    task="text-generation",
    max_new_tokens=150,
    do_sample=True,
    temperature=0.2,
)

model = ChatHuggingFace(llm=llm)

In [80]:
#Here is the embedding Logic
class GameEmbeddingModel:
    def __init__(self):
        self.model = SentenceTransformer("all-MiniLM-L6-v2")

    def embed_text(self, text: str) -> np.ndarray:
        return self.model.encode(text, normalize_embeddings=True)

    def embed_games(self, games: list[dict]) -> list[np.ndarray]:
        texts = [
            f"{game['title']} {' '.join(game['genre'])} {game['style']}"
            for game in games
        ]
        return self.model.encode(texts, normalize_embeddings=True)

In [79]:
#Loading the json file we are using as a database
def load_games(path="games.json"):
    with open(path, "r") as f:
        return json.load(f)


#Function for Cosine Similarity
def cosine_similarity(query_vec, game_vecs):
    return np.dot(game_vecs, query_vec)

In [78]:
#Scoring and ranking functions
def score_game(similarity : float, genre_match : float, popularity : float) -> float:
    return ((0.6 * similarity) + (0.3 * genre_match) + (0.1 * popularity))


def genre_match_score(game, task : str) -> float:
    task_lower = task.lower()
    matches = sum(1 for genre in game["genre"]
                  if genre.lower() in task_lower)
    return matches / len(game["genre"])

In [77]:
#Here we have the explanations how can we do that
EXPLAIN_PROMPT = """
You are an AI assistant explaining game recommendations.

User request:
{task}

Recommended games:
{games}

Rules:
- Explain each game briefly (2–3 lines max)
- Focus on genre, style, and why it matches the request
- Do NOT add new games
- Do NOT change the order
- Be concise and clear
"""



In [76]:
#Here we are going to make the state for our langgraph
class agentstate(TypedDict):
      messages : List[BaseMessage]
      task : str
      plan : Optional[list[str]]
      current_step : int
      result : Optional[str]
      decision : Optional[str]
      attempts : int
      error : Optional[str]

In [75]:
#Here we are going to make nodes
PLANNER_PROMPT = """
You are an AI planner for a game recommendation system.

Your job:
- Analyze the user request
- Decide if game recommendation is required
- Produce a clear, ordered plan

Rules:
- Output ONLY a numbered list of steps
- Use clear, concise steps
- If recommendations are needed, include steps for:
  - finding similar games
  - ranking games
  - explaining recommendations
- Do NOT execute anything
- Do NOT explain your reasoning

User request:
{task}
"""

def planner_node(state : agentstate) -> agentstate:
    prompt = PLANNER_PROMPT.format(task = state["task"])
    response = model.invoke(prompt)

    steps = [step.strip()
            for step in response.content.split("\n")
            if step.strip()
             ]

    state["plan"] = steps
    state["current_step"] = 0
    state["decision"] = "execute"
    return state

###############################################################
embedding_model = GameEmbeddingModel()
games = load_games()
game_embeddings = embedding_model.embed_games(games)

def executor_node(state : agentstate) -> agentstate:
    step = state["plan"][state["current_step"]].lower()
    SIMILARITY_THRESHOLD = 0.4

    if "similar" in step or "find" in step:
        query_vec = embedding_model.embed_text(state["task"])
        scores = cosine_similarity(query_vec, game_embeddings)
        ranked = []
        for i, sim in enumerate(scores):
            if sim < SIMILARITY_THRESHOLD:
                continue

            game = games[i]
            genre_score = genre_match_score(game, state["task"])
            final_score = score_game(
                similarity = sim,
                genre_match = genre_score,
                popularity = game["popularity"]
            )

            ranked.append((final_score, game))

        ranked.sort(reverse=True, key=lambda x: x[0])
        state["result"] = [game for _, game in ranked]

    state["current_step"] += 1
    return state

###########################################################

def validator_node(state : agentstate) -> agentstate:
    if not state.get("result"):
        state["decision"] = "end"
        return state

    if state["current_step"] < len(state["plan"]):
        state["decision"] = "continue"
        return state

    state["decision"] = "explain"
    return state


def build_graph():
    graph = StateGraph(agentstate)

    graph.add_node("planner", planner_node)
    graph.add_node("executor", executor_node)
    graph.add_node("validator", validator_node)
    graph.add_node("explainer", explainer_node)

    graph.add_edge(START, "planner")
    graph.add_edge("planner", "executor")
    graph.add_edge("executor", "validator")

    graph.add_conditional_edges(
        "validator",
        lambda state : state["decision"],
        {
            "continue" : "executor",
            "explain": "explainer",
            "end" : END
        }
    )

    graph.add_edge("explainer", END)

    return graph.compile()

def explainer_node(state: agentstate) -> agentstate:
    if not state.get("result"):
        state["result"] = "No recommendations could be generated."
        return state

    games = state["result"]

    games_text = "\n".join(
        f"- {game['title']} ({', '.join(game['genre'])}, {game['style']})"
        for game in state["result"]
    )

    prompt = EXPLAIN_PROMPT.format(
        task=state["task"],
        games=games_text
    )

    response = llm.invoke(prompt)

    state["result"] = response.content

    return state

In [85]:
graph = build_graph()

initial_state = {
    "messages" : [],
    "task" : "Recommend horror games like Resident Evil",
    "plan" : None,
    "current_step" : 0,
    "result" : None,
    "decision" : None,
    "attempts" : 0,
    "error" : None
}

output = graph.invoke(initial_state)

print("\nFinal Recommendation:\n")
print(output["result"])


Final Recommendation:

None
